# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp (°C),Humidity %,Cloudiness %,Wind Speed (m/s),Country,Date
0,0,berezovyy,51.6667,135.7000,-1.46,35,0,2.29,RU,1731469466
1,1,olonkinbyen,70.9221,-8.7187,-3.88,75,100,14.30,SJ,1731469467
2,2,margaret river,-33.9500,115.0667,22.60,51,47,4.46,AU,1731469469
3,3,grand falls-windsor,48.9332,-55.6649,5.28,92,100,2.68,CA,1731469325
4,4,waitangi,-43.9535,-176.5597,13.23,73,0,3.13,NZ,1731469471


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Longitude",  
    "Latitude", 
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity %", 
    scale=0.5,        # Adjust the scale as necessary
    color="City",
    hover_cols=['Country','Max Temp (°C)']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity %,Country,Max Temp (°C))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp (°C)'] > 20) &  # Example condition for temperature
    (city_data_df['Max Temp (°C)'] < 40) &  # Example condition for temperature
    (city_data_df['Humidity %'] < 60)       # Example condition for humidity
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Latitude,Longitude,Max Temp (°C),Humidity %,Cloudiness %,Wind Speed (m/s),Country,Date
2,2,margaret river,-33.9500,115.0667,22.60,51,47,4.46,AU,1731469469
9,9,tura,25.5198,90.2201,26.41,52,1,1.09,IN,1731469477
10,10,la foa,-21.7108,165.8276,28.49,53,82,0.90,NC,1731469478
16,16,hingorja,27.2109,68.4160,27.94,29,0,1.53,PK,1731469485
34,34,sangrur,30.2333,75.8333,25.00,36,0,1.19,IN,1731469506


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Latitude', 'Longitude', 'Humidity %', 'Max Temp (°C)', 'Wind Speed (m/s)']].copy()

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Latitude,Longitude,Humidity %,Max Temp (°C),Wind Speed (m/s),Hotel Name
2,margaret river,AU,-33.9500,115.0667,51,22.60,4.46,
9,tura,IN,25.5198,90.2201,52,26.41,1.09,
10,la foa,NC,-21.7108,165.8276,53,28.49,0.90,
16,hingorja,PK,27.2109,68.4160,29,27.94,1.53,
34,sangrur,IN,30.2333,75.8333,36,25.00,1.19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000  # Set the radius to 10,000 meters
params = {
    "categories": "accommodation.hotel",      # Specify the type of places to search for                     
    "limit": 1,    # Limit to 1 hotel
    "apiKey": geoapify_key  # Replace with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row['Latitude']
    longitude = row['Longitude']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Define the search area
    params["bias"] = f"proximity:{longitude},{latitude}"           # Bias towards the location

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make the API request

    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # Handle unsuccessful requests
        hotel_df.loc[index, "Hotel Name"] = "API request failed"
        print(f"Request failed with status code: {response.status_code}")
        print(f"Response content: {response.text}")
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
tura - nearest hotel: Hotel Polo Orchid
la foa - nearest hotel: Oasis de Tendea
hingorja - nearest hotel: No hotel found
sangrur - nearest hotel: Sunrize Palace
phek - nearest hotel: No hotel found
thoen - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
katra - nearest hotel: HOTEL OM HARI RESIDENCY
ke-macina - nearest hotel: No hotel found
jatiroto - nearest hotel: No hotel found
palangkaraya - nearest hotel: Swiss-Belhotel
ascencion de guarayos - nearest hotel: Hotel Ɨguasú
konarka - nearest hotel: Hotel Sun Village
ati - nearest hotel: No hotel found
shingu - nearest hotel: ホテルニューパレス
timbuktu - nearest hotel: Hotel La Maison
tsiroanomandidy - nearest hotel: Mega Food Hotel
kidal - nearest hotel: No hotel found
aioun - nearest hotel: فندق العيون
badiyah - nearest hotel: Al Areen Hotel Apartments
karratha - nearest hotel: Karratha International Hotel
banamba - nearest hotel: No

,City,Country,Latitude,Longitude,Humidity %,Max Temp (°C),Wind Speed (m/s),Hotel Name
2,margaret river,AU,-33.9500,115.0667,51,22.60,4.46,Margaret River Hotel
9,tura,IN,25.5198,90.2201,52,26.41,1.09,Hotel Polo Orchid
10,la foa,NC,-21.7108,165.8276,53,28.49,0.90,Oasis de Tendea
16,hingorja,PK,27.2109,68.4160,29,27.94,1.53,No hotel found
34,sangrur,IN,30.2333,75.8333,36,25.00,1.19,Sunrize Palace
37,phek,IN,25.6667,94.5000,53,22.19,0.56,No hotel found
54,thoen,TH,17.6129,99.2161,54,31.35,0.32,No hotel found
82,alice springs,AU,-23.7000,133.8833,26,29.75,4.63,Aurora Alice Springs
97,katra,IN,32.9833,74.9500,24,20.91,1.86,HOTEL OM HARI RESIDENCY
98,ke-macina,ML,13.9623,-5.3572,21,24.97,3.07,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map_df = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "City",
    hover_cols=['Hotel Name', 'Country']  # Add the columns for hotel name and country   
)

# Display the map
hotel_map_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Hotel Name,Country)